# Set up environment

| torch.__version__ | torchvision.__version__ | fastai.__version__ |                                                           |
| ----------------- | ----------------------- | ------------------ | --------------------------------------------------------- |
| 1.11.0            | 0.12.0                  | 2.5.6              | ✓                                                         |
| 1.9.1, 1.10.0     | 0.10.1                  | 2.5.3              | ✓                                                         |
| 1.6.0             | 0.7.0                   | 2.0.18             | ✗ (use [this](https://github.com/baidut/PatchVQ) instead) |



[Optional] Create a virtual environment

1. install [miniconda](https://docs.conda.io/en/latest/miniconda.html)
1. `conda create --name venv python=3.9`
1. `conda activate venv`

Install packages:

1. [Pytorch](https://pytorch.org/get-started/locally/) 
2. `pip install jupyter timm seaborn wandb logru fastai`


In [1]:
from fastiqa.basics import *

3.9.12 (main, Apr  5 2022, 06:56:58) 
[GCC 7.5.0]
fastai.__version__(>= 2.5.3): 2.5.6
fastcore.__version__: 1.4.2
torch.__version__(>= 1.9.1): 1.11.0 w/ cuda 
torchvision.__version__(>= 0.10.1): 0.12.0


# Test PatchVQ on your dataset

```
database_folder
├── labels.csv
├── dbinfo.json
├── jpg
│   ├── video1_image_folder
│   ├── video2_image_folder
│   └── ...
|
```

## Prepare dataset

1. Extract video frames
2. Under your dataset folder, put a json file (`dbinfo.json`) containing its key information:

```json
{
    "__name__": "LIVE_VQC",     # the name of the database
    "csv_labels": "labels.csv", # path to the label CSV file
    "fn_col": "name",           # filename column in the CSV table 
    "label_col": "mos",         # label column in the CSV table
    "folder": "jpg"             # the folder containing extracted video frames
}
```

## Extract features

In [ ]:
from fastiqa.patchvq.main import *
dbinfo = load_dbinfo("/path/to/dbinfo.json")

model = PatchVQ()
model.extractFeatures(dbinfo)

The extracted features will be saved under `features` folder

```
database_folder
├── labels.csv
├── dbinfo.json
├── jpg
│   ├── video1_image_folder
│   ├── video2_image_folder
│   └── ...
|
├── features  
│   ├── r3d18
│   └── r3d18_pooled
│   ├── paq2piq
│   └── paq2piq_pooled

```

## Extract Scores (coming soon):